## Controllo completezza Scraping

###Quanti articoli tra quelli scaricati sono poi stati effettivamente utilizzati? Sono stati scartati quelli senza twitter_link

In [ ]:
#@title
import pandas as pd
import numpy as np
import re
from google.colab import files
from google.colab import drive
import os
import glob
import json

# Colleghiamoci a Mydrive
drive.mount('/content/drive')

# Spostati nella cartella contenente tutti i dataset di scraping
# path = '/content/drive/MyDrive/Dataset/JSON SCRAPING'  
path1 = '/content/drive/MyDrive/DATA SCIENCE/Progetti I anno/Dataset/JSON SCRAPING'
path2 = '/content/drive/MyDrive/DATA SCIENCE/Progetti I anno/Dataset/JSON SCRAPING CLEAN'

os.chdir(path1)
file_paths1 = [h for h in glob.glob("*.json")]

os.chdir(path2)
file_paths2 = [i for i in glob.glob("*.json")]

for file_path in file_paths1:
  # print(re.sub(r' Twitter scraping_news.json', '', file_path))
  datas = {'Film' : [], 'Articoli trovati su Google news': [], 
            'Articoli utilizzati per scaricare tweets': [], 'Differenza': [],
            "Completezza rispetto all'attributo twitter_link (%)": []}
  try:
    file_json1 = open(path1 + '/' + file_path)
    file_json2 = open(path2 + '/' + re.sub(r".json", "_clean.json", file_path))
    
    data1 = json.load(file_json1)
    data2 = json.load(file_json2)

    datas['Film'].append(str(re.sub(r' Twitter scraping_news.json', '', file_path)))
    datas['Articoli trovati su Google news'].append(int(len(data1)))
    datas['Articoli utilizzati per scaricare tweets'].append(int(len(data2)))
    datas['Differenza'].append(int(len(data1)-len(data2)))
    datas["Completezza rispetto all'attributo twitter_link (%)"].append(str(len(data2)/len(data1)*100))

    a = pd.DataFrame(datas)
    a.to_csv('/content/drive/MyDrive/DATA SCIENCE/Progetti I anno/Tabelle Report/{}_completezza.csv'.format(file_path.strip(' Twitter scraping_news.json')))
    file_json1.close()
    file_json2.close()
  except:
    b = pd.DataFrame(columns=['Film', 'Articoli trovati su Google news', 'Articoli utilizzati per scaricare tweets', 'Differenza', "Completezza rispetto all'attributo twitter_link (%)"])
    b['Film'] = file_path.strip(' Twitter scraping_news.json')
    b.to_csv('/content/drive/MyDrive/DATA SCIENCE/Progetti I anno/Tabelle Report/{}_completezza.csv'.format(file_path.strip(' Twitter scraping_news.json')))
    continue


In [ ]:
#@title
path = '/content/drive/MyDrive/DATA SCIENCE/Progetti I anno/Tabelle Report'
os.chdir(path)

dir = [i for i in glob.glob("*_completezza.csv")]

a0 = pd.read_csv(path + '/' + dir[20], index_col = 'Unnamed: 0')
for file in dir[1:]:
  a1 = pd.read_csv(path + '/' + file, index_col = 'Unnamed: 0')
  a0 = pd.concat([a0, a1], ignore_index=True)
a0.to_csv('/content/drive/MyDrive/DATA SCIENCE/Progetti I anno/Tab_quality_assessment/Tab_Completezza_Scraping.csv')
a0

## Controllo completezza Twitter

###Quanti tweet sono stati scaricati rispetto a quelli disponibili?

In [ ]:
#@title
import pandas as pd
import numpy as np
import re
from google.colab import files
from google.colab import drive
import os
import glob
import json

# Colleghiamoci a Mydrive
drive.mount('/content/drive')

sc = '/content/drive/MyDrive/DATA SCIENCE/Progetti I anno/Dataset/JSON SCRAPING CLEAN'
os.chdir(sc)

file = [i for i in glob.glob("*.json")]

tw = '/content/drive/MyDrive/DATA SCIENCE/Progetti I anno/Dataset/JSON TWITTER'
os.chdir(tw)

file2 = [h for h in glob.glob("*.json")]

data = {'Film': [], 'N° tweet_id estratti': [], 'N° tweet_id trovati': [], 'Differenza': [], 'Completezza in ipotesi di mondo aperto (rispetto a tutti i tweets estraibili)(%)': []}
for f in file2:
  data['Film'].append(re.sub(' Twitter.json', '', f).strip('Final_post_'))
  
  tw_film = json.loads(open(tw + '/' + f).read())
  g = f.split('Final_post_')[1]
  sc_film = json.loads(open(sc + '/' + re.sub('.json', ' scraping_news_clean.json', g)).read())
  n_tw_sc = 0
  for el in range(len(sc_film)):
    n_tw_sc = n_tw_sc + len(sc_film[el]['twitter_id'])

  data['N° tweet_id estratti'].append(n_tw_sc)  
  data['N° tweet_id trovati'].append(len(tw_film))  
  data['Differenza'].append(n_tw_sc-len(tw_film))  
  data['Completezza in ipotesi di mondo aperto (rispetto a tutti i tweets estraibili)(%)'].append((len(tw_film)/n_tw_sc)*100)  

a = pd.DataFrame(data) 
a.to_csv('/content/drive/MyDrive/DATA SCIENCE/Progetti I anno/Tab_quality_assessment/Tab_Completezza_Mondo_Aperto.csv')
a

## Controllo completezza attributi twitter

###Quanto sono completi i dataset contenenti i tweets?

In [ ]:
#@title
#Per ogni dataset, per ogni colonna guardo quanti sono le righe vuote / nulle fratto quelle totali

import pandas as pd
import numpy as np
import re
from google.colab import files
from google.colab import drive
import os
import glob
import math
import json

# Colleghiamoci a Mydrive
drive.mount('/content/drive')

# Spostati nella cartella contenente tutti i dataset di scraping
path = '/content/drive/MyDrive/DATA SCIENCE/Progetti I anno/Dataset/3)TWEETS API'

os.chdir(path)
file_paths = [h for h in glob.glob("*.json")]

datas = {'film':[], 'hasthag_not_empty %': [], 'url_not_empty %': [], 
            'username_not_empty %': [], 'User_id_not_empty %': [],
            "User_verified_not_empty %": [],"User_creation_not_empty %": [],"Tweet_id_not_empty %": [],"Date_not_empty %": [],
           "User_Description_not_empty %": [],"Text_not_empty %": [],"Source_not_empty %": []}
  
for file_path in file_paths:
  try:
    file_json = open(path + '/' + file_path)
    data = json.load(file_json)
    count_hasthag_not_empty=0
    count_url_not_empty=0
    count_username_not_empty=0
    count_User_id_not_empty=0
    count_User_verified_not_empty=0
    count_User_creation_not_empty=0
    count_Tweet_id_not_empty=0
    count_Date_not_empty=0
    count_User_Description_not_empty=0
    count_Text_not_empty=0
    count_Source_not_empty=0
    for tweet in data:
      if len(tweet['Hashtags'])!=0:
        count_hasthag_not_empty +=1
      if len(tweet['Urls'])!=0:
        count_url_not_empty +=1
      if tweet['Username']!="" and tweet['Username']!=None:
        count_username_not_empty+=1
      if tweet['User_id']!="" and math.isnan(tweet['User_id'])==False:
        count_User_id_not_empty+=1
      if tweet['User_verified']!="" and math.isnan(tweet['User_verified'])==False:
        count_User_verified_not_empty+=1
      if tweet['User_creation']!="" and tweet['User_creation']!=None:
        count_User_creation_not_empty+=1
      if tweet['Tweet_id']!="" and tweet['Tweet_id']!=None:
        count_Tweet_id_not_empty+=1
      if tweet['Date']!="" and tweet['Date']!=None:
        count_Date_not_empty+=1
      if tweet['User_Description']!="" and tweet['User_Description']!=None:
        count_User_Description_not_empty+=1
      if tweet['Text']!="" and tweet['Text']!=None:
        count_Text_not_empty+=1
      if tweet['Source']!="" and tweet['Source']!=None:
        count_Source_not_empty+=1
    
    datas['film'].append(str(file_path.strip(' Twitter.json').strip('Final_post_')))
    datas['hasthag_not_empty %'].append(str(count_hasthag_not_empty/len(data)*100))
    datas['url_not_empty %'].append(str(count_url_not_empty/len(data)*100))
    datas['username_not_empty %'].append(str(count_username_not_empty/len(data)*100))
    datas['User_id_not_empty %'].append(str(count_User_id_not_empty/len(data)*100))
    datas['User_verified_not_empty %'].append(str(count_User_verified_not_empty/len(data)*100))
    datas['User_creation_not_empty %'].append(str(count_User_creation_not_empty/len(data)*100))
    datas['Tweet_id_not_empty %'].append(str(count_Tweet_id_not_empty/len(data)*100))
    datas['Date_not_empty %'].append(str(count_Date_not_empty/len(data)*100))
    datas['User_Description_not_empty %'].append(str(count_User_Description_not_empty/len(data)*100))
    datas['Text_not_empty %'].append(str(count_Text_not_empty/len(data)*100))
    datas['Source_not_empty %'].append(str(count_Source_not_empty/len(data)*100))
    
  except:
    bf = file_path.strip(' Twitter.json').strip('Final_post_')
    datas['film'].append(bf)
    datas['hasthag_not_empty %'].append('0')
    datas['url_not_empty %'].append('0')
    datas['username_not_empty %'].append('0')
    datas['User_id_not_empty %'].append('0')
    datas['User_verified_not_empty %'].append('0')
    datas['User_creation_not_empty %'].append('0')
    datas['Tweet_id_not_empty %'].append('0')
    datas['Date_not_empty %'].append('0')
    datas['User_Description_not_empty %'].append('0')
    datas['Text_not_empty %'].append('0')
    datas['Source_not_empty %'].append('0')
    continue
  
a = pd.DataFrame(datas)
a.to_csv('/content/drive/MyDrive/DATA SCIENCE/Progetti I anno/Tab_quality_assessment/Tab_Completezza_Attributi_Tweets.csv')
a.head(5)

###CONSISTENZA

####Controllo che data pubblicazione tweet (Date) deve essere successiva a data creazione utente (User_creation), altrimenti c'è un'inconsistenza in logica

In [ ]:
#@title
# Le date di pubblicazione del tweet (twitter_id.Date) devonono essere precedenti alla data di pubblicazione dell'articolo (published)
 
import pandas as pd
import numpy as np
import re
from google.colab import files
from google.colab import drive
import os
import glob
import json

# Colleghiamoci a Mydrive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/DATA SCIENCE/Progetti I anno/Dataset/Mongodb DF (JSON)'
os.chdir(path)

file = [i for i in glob.glob("*.json")]

print('Film che presentano inconsistenza logica tra data creazione utente e data di pubblicazione del tweet')
for f in file:
  film = json.loads(open(path + '/' + f).read())
## Controlliamo che Date >= User_creation 
  try:
    for art_tw in film['Articles and Tweets']:
      err_ut_tw = []
      ut_date = []
      tw_date = []
      ut_err = []
      err_ut_tw_count = 0
      for tw in art_tw['twitter_id']:
        anno_tw = int(tw['Date'].split('/')[2])
        mese_tw = int(tw['Date'].split('/')[1])
        giorno_tw = int(tw['Date'].split('/')[0])
        
        anno_ut = int(tw['User_creation'].split('/')[2])
        mese_ut = int(tw['User_creation'].split('/')[1])
        giorno_ut = int(tw['User_creation'].split('/')[0])
        
        # Caso 1: Utente e tweet differiscono per qualche giorno 
        if anno_tw == anno_ut:
          if mese_tw == mese_ut:
            if giorno_tw < giorno_ut:
              # print('Errore: data creazione utente posteriore a data pubblicazione tweet')
              err_ut_tw.append(tw['Tweet_id'])
              ut_date.append(tw['User_creation'])
              tw_date.append(tw['Date'])
              ut_err.append(tw['User_id'])
              err_ut_tw_count += 1
            else:
              # print('ok')
              continue
          else:
            continue
        else:
          continue
        
        # Caso 2: Utente e tweet differiscono per qualche mese
        if anno_tw == anno_ut:
          if mese_tw < mese_ut:
              # print('Errore: data creazione utente posteriore a data pubblicazione tweet')
              err_ut_tw.append(tw['Tweet_id'])
              ut_date.append(tw['User_creation'])
              tw_date.append(tw['Date'])
              ut_err.append(tw['User_id'])
              err_ut_tw_count += 1
          else:
            continue
        else:
          continue

        # Caso 3: Utente e tweet differiscono per qualche anno 
        if anno_tw < anno_ut:
          err_ut_tw.append(tw['Tweet_id'])
          ut_date.append(tw['User_creation'])
          tw_date.append(tw['Date'])
          ut_err.append(tw['User_id'])
          err_ut_tw_count += 1
        else:
          continue

    if err_ut_tw_count != 0: 
      print(re.sub('.json', '', f), 'n° inconsistenze utente-tweet = ', err_ut_tw_count, '; Tweet_id inconsistenti = ', err_ut_tw, 
            '; User_id inconsistenti = ', ut_err, 'Data creazione utente:', ut_date, 'Data pubblicazione tweet:', tw_date)
    else:
      print(re.sub('.json', '', f), 'Documento consistente')
  except:
    # print('Articles and Tweets empty')
    continue


####Controllo data pubblicazione tweet (Date) deve essere precedente a data pubblicazione articolo (published), altrimenti c'è un'inconsistenza in logica

In [ ]:
#@title
import pandas as pd
import numpy as np
import re
from google.colab import files
from google.colab import drive
import os
import glob
import json

# Colleghiamoci a Mydrive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/DATA SCIENCE/Progetti I anno/Dataset/Mongodb DF (JSON)'
os.chdir(path)

file = [i for i in glob.glob("*.json")]

print('Film che presentano inconsistenza logica tra data di pubblicazione del tweet e data pubblicazione articolo')

for f in file:
  film = json.loads(open(path + '/' + f).read())
## Controlliamo che Date <= published 
  try:
  # if film['Articles and Tweets'] == film['Articles and Tweets']:
    data = {'Film': [], 'art_date': [], 'art_date' : [], 'err_art_tw' : [], 'tw_date' : []}

    for art_tw in film['Articles and Tweets']:
      anno_art = int(art_tw['published'].split('/')[2])
      mese_art = int(art_tw['published'].split('/')[1])
      giorno_art = int(art_tw['published'].split('/')[0])
        
      for tw in art_tw['twitter_id']:
        anno_tw = int(tw['Date'].split('/')[2])
        mese_tw = int(tw['Date'].split('/')[1])
        giorno_tw = int(tw['Date'].split('/')[0])

        if anno_art == anno_tw:
          if mese_art == mese_tw:
            if giorno_art < giorno_tw:
              data['Film'].append(f.strip('.json'))
              data['err_art_tw'].append(tw['Tweet_id'])
              data['art_date'].append(art_tw['published'])
              data['tw_date'].append(tw['Date'])
            else:
              continue    
          if mese_art < mese_tw:
            data['Film'].append(f.strip('.json'))
            data['err_art_tw'].append(tw['Tweet_id'])
            data['art_date'].append(art_tw['published'])
            data['tw_date'].append(tw['Date'])
          else:
            # print('ok')
            continue
        if anno_art < anno_tw:
          data['Film'].append(f.strip('.json'))
          data['err_art_tw'].append(tw['Tweet_id'])
          data['art_date'].append(art_tw['published'])
          data['tw_date'].append(tw['Date'])
        else:
          # print('ok2')
          continue
      
    if len(data) != 0:
      a = pd.DataFrame(data)
      a.to_csv('/content/drive/MyDrive/DATA SCIENCE/Progetti I anno/Tabelle Report/{}x.csv'.format(f.strip('.json')))
      # print(a)
    else:
      b = pd.DataFrame(columns=['Film', 'art_date', 'err_art_tw', 'tw_date'])
      b['Film'] = f.strip('.json')
      b.to_csv('/content/drive/MyDrive/DATA SCIENCE/Progetti I anno/Tabelle Report/{}x.csv'.format(f.strip('.json')))
      # print(f.strip('.json'), 'Documento consistente')
  except:
    c = pd.DataFrame(columns=['Film', 'art_date', 'err_art_tw', 'tw_date'])
    c['Film'] = f.strip('.json')
    c.to_csv('/content/drive/MyDrive/DATA SCIENCE/Progetti I anno/Tabelle Report/{}x.csv'.format(f.strip('.json')))
    continue

In [ ]:
#@title
path = '/content/drive/MyDrive/DATA SCIENCE/Progetti I anno/Tabelle Report'
os.chdir(path)

dir = [i for i in glob.glob("*x.csv")]

a0 = pd.read_csv(path + '/' + dir[0], index_col = 'Unnamed: 0')
for file in dir[1:]:
  a1 = pd.read_csv(path + '/' + file, index_col = 'Unnamed: 0')
  a0 = pd.concat([a0, a1], ignore_index=True)

a0.rename(columns = {'art_date': 'Data pubblicazione articolo', 'err_art_tw': 'Tweet_id', 'tw_date': 'Data pubblicazione tweet'}, inplace = True)
a0.to_csv('/content/drive/MyDrive/DATA SCIENCE/Progetti I anno/Tab_quality_assessment/Tab_Consistenza_tra_DataArticolo_DataTweet.csv')
a0.groupby('Film', as_index = False).count().rename(columns={'Tweet_id': 'N° inconsistenze'})[['Film', 'N° inconsistenze']]

In [ ]:
#@title
a0.head(5)

## Controllo consistenza per unicità tweet id

###Controlliamo che non ci siano tweet_id duplicati

In [ ]:
#@title
import pandas as pd
import numpy as np
import re
from google.colab import files
from google.colab import drive
import os
import glob
import math
import json

# Colleghiamoci a Mydrive
drive.mount('/content/drive')

# Spostati nella cartella contenente tutti i dataset di scraping
path = '/content/drive/MyDrive/DATA SCIENCE/Progetti I anno/Dataset/4)DOCUMENTS MONGODB'

os.chdir(path)
file_paths = [h for h in glob.glob("*.json")]

df = {'Film': [], 'N° Inconsistenze': [], 'Totale': [], 'Rapporto N° Inconsistenze/Totale(%)': []}

for file_path in file_paths:
  file_json = open(path + '/' + file_path)
  data = json.load(file_json)
  diz_id_user={}
  list_tweet_id =[]
  count_duplicate_tweet_id=0
  count_inconsistency=0
  count_tot=0
  if data['Articles and Tweets']==data['Articles and Tweets']:
    if tweet['Tweet_id'] not in diz_id_user:
      diz_id_user[tweet['Tweet_id']]=tweet['Username']
    else:
      if diz_id_user[tweet['Tweet_id']]!=tweet['Username']:
        count_inconsistency+=1
    count_tot+=1
  
  df['Film'].append(re.sub('.json', '', file_path).strip('Final_post_'))
  df['N° Inconsistenze'].append(count_inconsistency)
  df['Totale'].append(count_tot)
  try:
    df['Rapporto N° Inconsistenze/Totale(%)'].append(str(count_inconsistency/count_tot*100))
  except:
    df['Rapporto N° Inconsistenze/Totale(%)'].append(np.nan)
    continue

data = pd.DataFrame(df)
data.to_csv('/content/drive/MyDrive/DATA SCIENCE/Progetti I anno/Tab_quality_assessment/Unicità_Tweet_id.csv')
a